In [18]:
import pandas as pd
import requests
import time

# File paths (Update as needed)
file_path = "/Users/elliehozhabri/Documents/RBP/scrape/UTSW_OpenAlexInput.xlsx"
output_file_path = "/Users/elliehozhabri/Documents/RBP/scrape/UTSW_OpenAlexResults.xlsx"

# Load the original spreadsheet
df_original = pd.read_excel(file_path)

# Create a new DataFrame with selected columns
columns_to_copy = ["University", "First", "Middle", "Last", "Nickname"]
df_results = df_original[columns_to_copy].copy()

# Add new OpenAlex-related columns
df_results["OpenAlexLink"] = "N/A"
df_results["OpenAlexID"] = "N/A"
df_results["Name Verification"] = "N/A"
df_results["Institute Verification"] = "N/A"
df_results["sameUni"] = "N/A"  # New column to check if institutions match

# OpenAlex API URL
openalex_api_url = "https://api.openalex.org/authors"

def search_openalex(first, last, middle="", nickname=""):
    """Search for an author in OpenAlex using provided names, considering alternative names."""
    query_names = [f"{first} {last}"]
    
    if pd.notna(middle) and middle.strip():
        query_names.append(f"{first} {middle} {last}")
    if pd.notna(nickname) and nickname.strip():
        query_names.append(f"{nickname} {last}")

    for query in query_names:
        try:
            response = requests.get(openalex_api_url, params={"search": query, "per_page": 25}, timeout=10)
            if response.status_code == 200:
                results = response.json().get("results", [])
                return results
        except requests.exceptions.RequestException as e:
            print(f"Error searching OpenAlex for {query}: {e}")
        time.sleep(1)  # Prevent rate limiting
    return []

# Process only the first 10 rows for debugging
for index, row in df_results.iterrows():  # Process only 10 rows
    first, last = row["First"], row["Last"]
    middle = row.get("Middle", "")
    nickname = row.get("Nickname", "")
    university = row["University"]  # University name from original data

    if pd.isna(first) or pd.isna(last):
        continue  # Skip rows with missing names

    print(f"Searching OpenAlex for: {first} {last}")  # Debugging print statement

    results = search_openalex(first, last, middle, nickname)

    if results:
        best_match = None
        best_score = -1  # Track best match score

        for res in results:
            score = 0  # Ranking system

            # Check exact name match (Display Name)
            if res["display_name"].lower() == f"{first} {last}".lower():
                score += 3  # Strong match
            elif middle and res["display_name"].lower() == f"{first} {middle} {last}".lower():
                score += 2  # Middle name match
            elif nickname and res["display_name"].lower() == f"{nickname} {last}".lower():
                score += 1  # Nickname match

            # Check Alternate Names
            if "display_name_alternatives" in res:
                alt_names = [name.lower() for name in res["display_name_alternatives"]]
                if f"{first} {last}".lower() in alt_names:
                    score += 3  # Strong match on alternate name
                elif middle and f"{first} {middle} {last}".lower() in alt_names:
                    score += 2
                elif nickname and f"{nickname} {last}".lower() in alt_names:
                    score += 1

            # Extract all current institutions
            institutions = [inst["institution"]["display_name"] for inst in res.get("affiliations", []) if "institution" in inst]
            past_institutions = [inst["display_name"] for inst in res.get("last_known_institutions", [])] if "last_known_institutions" in res else []

            # Prioritize The University of Texas Southwestern Medical Center (UTSW) even more
            if "The University of Texas Southwestern Medical Center" in institutions:
                score += 7  # High priority for UTSW in current institutions
            elif "The University of Texas Southwestern Medical Center" in past_institutions:
                score += 5  # Moderate priority if UTSW is in past institutions

            # Use OpenAlex's relevance score as a tie-breaker, but with lower weight
            score += res.get("relevance_score", 0) / 2000  # Normalize score, less influence

            if score > best_score:
                best_score = score
                best_match = res  # Select the best-scoring profile

        # If no match is found, fall back to the first result
        if not best_match and results:
            best_match = results[0]

        # Extract OpenAlex ID and verified name
        author_id = best_match["id"].split("/")[-1]
        author_name = best_match["display_name"]

        # Extract all current institutions (before "Past Institutions")
        institutions = []

        if "affiliations" in best_match and best_match["affiliations"]:
            for aff in best_match["affiliations"]:
                if "institution" in aff and "id" in aff["institution"] and "display_name" in aff["institution"]:
                    inst_name = aff["institution"]["display_name"]
                    institutions.append(inst_name)

        # Convert list to a comma-separated string, or set to "N/A" if empty
        institutions_str = ", ".join(institutions) if institutions else "N/A"

        # Compare with the "University" column and set "sameUni" to Y or N
        same_uni = "Y" if "The University of Texas Southwestern Medical Center" in institutions_str or \
                          "The University of Texas Southwestern Medical Center" in past_institutions else "N"

        # Store extracted data
        df_results.at[index, "OpenAlexLink"] = f"https://openalex.org/authors/{author_id}"
        df_results.at[index, "OpenAlexID"] = author_id
        df_results.at[index, "Name Verification"] = author_name
        df_results.at[index, "Institute Verification"] = institutions_str
        df_results.at[index, "sameUni"] = same_uni  # Y/N check for university match

    else:
        print(f"No OpenAlex match found for {first} {last}")  # Debugging print statement

# Save the new spreadsheet
df_results.to_excel(output_file_path, index=False)

print(f"Debugging complete. Results saved to: {output_file_path}")


Searching OpenAlex for: Kareem AbdelFattah
Searching OpenAlex for: Waleed Abdellatif
Searching OpenAlex for: Shuaib Abdullah
Searching OpenAlex for: Osama Aboul Fettouh
No OpenAlex match found for Osama Aboul Fettouh
Searching OpenAlex for: Carol Abousaab
Searching OpenAlex for: Annie Abraham
Searching OpenAlex for: Geoffrey Abraham
Searching OpenAlex for: Jocelyn Abraham
Searching OpenAlex for: Leah Abraham
Searching OpenAlex for: Reeni Abraham
Searching OpenAlex for: Jessica Abramowitz
Searching OpenAlex for: John Abrams
Searching OpenAlex for: Marconi Abreu
Searching OpenAlex for: Elie Abu Jawdeh
Searching OpenAlex for: Muhanned Abu-Hijleh
Searching OpenAlex for: Hussna Abunafeesa
Searching OpenAlex for: Hila Abush Segev
No OpenAlex match found for Hila Abush Segev
Searching OpenAlex for: Asha Acharya
Searching OpenAlex for: Samuel Achilefu
Searching OpenAlex for: Quentin Adams
Searching OpenAlex for: Richard Adams
Searching OpenAlex for: Traci Adams
Searching OpenAlex for: William 